## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/gbdt_pipeline')
from gbdt_pipeline import get_gbdt_pipeline, train_gbdt, train_gbdt_complete

current path /Users/JUSC/Documents/xgbsurv/experiments/ah


## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 10
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_gbdt_'


# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'ah'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
Fitting 5 folds for each of 10 candidates, totalling 50 fits
integration values 179198
integration values 31396
Concordance Index 0.5943216539026229
Integrated Brier Score: 0.19289044826098936
{'model': 'ah', 'dataset': 'METABRIC', 'cindex_train': [0.6013024782816181], 'cindex_test': [0.5943216539026229], 'ibs_train': [0.17735107355946655], 'ibs_score_test': [0.19289044826098936]}
1
Fitting 5 folds for each of 10 candidates, totalling 50 fits
integration values 167931
integration values 29618
Concordance Index 0.5572774292513127
Integrated Brier Score: 0.18578269722467403
{'model': 'ah', 'dataset': 'METABRIC', 'cindex_train': [0.5739909808766047], 'cindex_test': [0.5572774292513127], 'ibs_train': [0.1794880075358399], 'ibs_score_test': [0.18578269722467403]}
2
Fitting 5 folds for each of 10 candidates, totalling 50 fits
integration values 172762
integration values 33101
Concordance Index 0.5545665404515988
Integrated Brier Score: 0.19564685846348134
{'model': 'ah

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_score_test
0,ah,METABRIC,0.601302,0.594322,0.177351,0.192890
1,ah,METABRIC,0.573991,0.557277,0.179488,0.185783
2,ah,METABRIC,0.595637,0.554567,0.178181,0.195647
3,ah,METABRIC,0.595177,0.563738,0.181970,0.177587
4,ah,METABRIC,0.600171,0.572303,0.179558,0.181095
5,ah,FLCHAIN,0.114958,0.090113,0.132353,0.136267
6,ah,FLCHAIN,0.737057,0.710709,0.127377,0.126838
7,ah,FLCHAIN,0.689810,0.676883,0.127783,0.129397
8,ah,FLCHAIN,0.745069,0.707220,0.125365,0.124414
9,ah,FLCHAIN,0.758010,0.758915,0.132192,0.128131


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'ah'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
Fitting 5 folds for each of 10 candidates, totalling 50 fits
integration values 1314
integration values 247
Concordance Index 0.5961325966850829
Integrated Brier Score: 0.23260708082092107
{'model': 'ah', 'dataset': 'BLCA', 'cindex_train': [0.5738412183340094], 'cindex_test': [0.5961325966850829], 'ibs_train': [0.21195899452259956], 'ibs_score_test': [0.23260708082092107]}
1
Fitting 5 folds for each of 10 candidates, totalling 50 fits
integration values 19228
integration values 1947
Concordance Index 0.5385035499726926
Integrated Brier Score: 0.23638855116097351
{'model': 'ah', 'dataset': 'BLCA', 'cindex_train': [0.659580782786125], 'cindex_test': [0.5385035499726926], 'ibs_train': [0.1760130512849949], 'ibs_score_test': [0.23638855116097351]}
2
Fitting 5 folds for each of 10 candidates, totalling 50 fits
integration values 10375
integration values 1548
Concordance Index 0.4050179211469534
Integrated Brier Score: 0.21828877288083484
{'model': 'ah', 'dataset': 'BLCA',

In [8]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)

In [9]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_score_test
0,ah,BLCA,0.573841,0.596133,0.211959,0.232607
1,ah,BLCA,0.659581,0.538504,0.176013,0.236389
2,ah,BLCA,0.585469,0.405018,0.200013,0.218289
3,ah,BLCA,0.648367,0.533219,0.186958,0.243553
4,ah,BLCA,0.577080,0.437621,0.204463,0.218299
5,ah,BRCA,0.562690,0.251736,0.190496,0.213051
6,ah,BRCA,0.563767,0.431309,0.194952,0.198514
7,ah,BRCA,0.680494,0.404104,0.182861,0.203223
8,ah,BRCA,0.641496,0.456257,0.196695,0.183274
9,ah,BRCA,0.248671,0.118796,0.196803,0.227560
